In [50]:
using RigidBodyDynamics
using LinearAlgebra
using StaticArrays
using MeshCat
using MeshCatMechanisms
using MechanismGeometries
using CoordinateTransformations
using GeometryBasics

include("/home/hkolano/onr-dynamics-julia/simulate_with_ext_forces.jl")
println("Libraries imported.")

Libraries imported.


In [51]:
# ------------------------------------------------------------------------
#                            MODEL DEFINITION
# ------------------------------------------------------------------------
vis = Visualizer()
render(vis)
# Define paths to the URDF files
src_dir = dirname(pathof(RigidBodyDynamics))
urdf_file_alpha = joinpath(src_dir, "..", "..", "..", "..", "..", "onr-dynamics-julia", "URDFs", "alphaArmFixedJaws.urdf")
urdf_file_water = joinpath(src_dir, "..", "..", "..", "..", "..", "onr-dynamics-julia", "URDFs", "alphaArmWater.urdf")
mechanism_alpha = parse_urdf(urdf_file_alpha)
mechanism_water = parse_urdf(urdf_file_water; gravity = [0.0, 0.0, 0.0])

delete!(vis)

# Create visuals of the URDFs
visuals = URDFVisuals(urdf_file_alpha)
water_vis = URDFVisuals(urdf_file_water)
mvis_alpha = MechanismVisualizer(mechanism_alpha, URDFVisuals(urdf_file_alpha), vis[:alpha])
mvis_water = MechanismVisualizer(mechanism_water, URDFVisuals(urdf_file_water))
# print(typeof(mvis_alpha))

# Name the joints of the mechanisms
base_joint, shoulder_joint, elbow_joint, wrist_joint = joints(mechanism_alpha)
base_j_water, shoulder_j_water, elbow_j_water, wrist_j_water = joints(mechanism_water)
~, shoulder_body, upper_arm_body, elbow_body, wrist_body = bodies(mechanism_alpha)

shoulder_frame = default_frame(shoulder_body)
upper_arm_frame = default_frame(upper_arm_body)
elbow_frame = default_frame(elbow_body)
wrist_frame = default_frame(wrist_body)
base_frame = root_frame(mechanism_alpha)
# # last_link = bodies(mechanism_alpha)[end]
# # body_frame = default_frame(last_link)

# Show default coordinate frames of these frames
# setelement!(mvis_alpha, base_frame)
# # setelement!(mvis_alpha, base_joint_frame)
# setelement!(mvis_alpha, shoulder_frame)
# # setelement!(mvis_alpha, elbow_frame)
# setelement!(mvis_alpha, wrist_frame)

arrow_buoy_og = ArrowVisualizer(vis[:arrow1])
setobject!(arrow_buoy_og)
arrow_buoy_trans = ArrowVisualizer(vis[:arrow2])
setobject!(arrow_buoy_trans)

println("URDFs parsed. Visualizers ready. \n")

URDFs parsed. Visualizers ready. 



┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8706
└ @ MeshCat /home/hkolano/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8707
└ @ MeshCat /home/hkolano/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


In [52]:
# ------------------------------------------------------------------------
#                          INITIAL CONDITIONS
# ------------------------------------------------------------------------
function impose_alpha_state_on_water(state_alpha, state_water)
    """
    Take the current state (configuration and velocity) of the alpha arm and impose it on the water mechanism state.
    """
    set_configuration!(state_water, base_j_water, configuration(state_alpha, base_joint))
    set_configuration!(state_water, shoulder_j_water, configuration(state_alpha, shoulder_joint))
    set_configuration!(state_water, elbow_j_water, configuration(state_alpha, elbow_joint))
    set_configuration!(state_water, wrist_j_water, configuration(state_alpha, wrist_joint))
    set_velocity!(state_water, base_j_water, velocity(state_alpha, base_joint))
    set_velocity!(state_water, shoulder_j_water, velocity(state_alpha, shoulder_joint))
    set_velocity!(state_water, elbow_j_water, velocity(state_alpha, elbow_joint))
    set_velocity!(state_water, wrist_j_water, velocity(state_alpha, wrist_joint))
end;

function impose_alpha_desacc_on_water(des_acc, des_acc_water)
    des_acc_water[base_j_water][1] = des_acc[base_joint][1]
    des_acc_water[shoulder_j_water][1] = des_acc[shoulder_joint][1]
    des_acc_water[elbow_j_water][1] = des_acc[elbow_joint][1]
    des_acc_water[wrist_j_water][1] = des_acc[wrist_joint][1]
end;

# Initialize state of alpha arm
state_alpha = MechanismState(mechanism_alpha)
zero!(state_alpha)
# set_configuration!(state, base_joint, 0.0)
# set_configuration!(state, shoulder_joint, -.27079)
# set_configuration!(state, elbow_joint, 3.1415/2+1.3)
# set_configuration!(state, wrist_joint, 0.5)

# Initialize state of water arm
state_water = MechanismState(mechanism_water)
# Make water state = arm state
impose_alpha_state_on_water(state_alpha, state_water)
# state_water_no_vel = deepcopy(state_water)

des_acc = similar(velocity(state_alpha))
des_acc[base_joint][1] = 0.5
des_acc[shoulder_joint][1] = 0.
des_acc[elbow_joint][1] = 0.
des_acc[wrist_joint][1] = 0.

des_acc_water = similar(velocity(state_water))
impose_alpha_desacc_on_water(des_acc, des_acc_water)

0.0

In [53]:
# ------------------------------------------------------------------------
#                              CONTROLLER
# ------------------------------------------------------------------------

function simple_control!(torques::AbstractVector, t, state_alpha::MechanismState)
    # Calculate inverse dynamics of alpha arm
    tau = inverse_dynamics(state_alpha, des_acc)
    # Find dynamic terms for water model
    impose_alpha_state_on_water(state_alpha, state_water)
    tau_water = inverse_dynamics(state_water, des_acc_water)

    # Set torques accordingly (controller)
    torques[velocity_range(state_alpha, base_joint)] .= tau[base_joint][1]
    torques[velocity_range(state_alpha, shoulder_joint)] .= tau[shoulder_joint][1]
    torques[velocity_range(state_alpha, elbow_joint)] .= tau[elbow_joint][1]
    torques[velocity_range(state_alpha, wrist_joint)] .= tau[wrist_joint][1]
end;

In [54]:
# ------------------------------------------------------------------------
#                      HYDRODYNAMICS CALCULATOR
# ------------------------------------------------------------------------
function hydro_calc!(hydro_wrenches::Dict, t, state_alpha::MechanismState)
    base_frame = root_frame(mechanism_alpha)
    wrench_test = Wrench(default_frame(findbody(mechanism_alpha, "r5m_shoulder_link")), [1.0, 0.0, 0.0], [0.0, 0.0, 0.0])
    
    hydro_wrenches = Dict{BodyID, Wrench}(
        BodyID(findbody(mechanism_alpha, "r5m_shoulder_link")) => transform(state_alpha, wrench_test, base_frame)
    )
end;

In [59]:
shoulder_ID = BodyID(shoulder_body)

println(default_frame(wrist_body))
println(body_fixed_frame_to_body(mechanism_alpha, default_frame(wrist_body)))

transform_from_base_to_wrist = relative_transform(state_alpha, root_frame(mechanism_alpha), default_frame(wrist_body))
# println(transform_from_base_to_wrist)

wrist_buoy_linear = FreeVector3D(base_frame, [0.0, 0.0, .2])
wrist_buoy_transformed = transform(state_alpha, wrist_buoy_linear, default_frame(wrist_body))
println(wrist_buoy_transformed.v)
# angular = FreeVector3D(base_frame, [0.0, 0.0, 0.0])

settransform!(arrow_buoy_og, Point(0.0, 0.0, 0.0), Vec{3}(wrist_buoy_linear.v))
pt = Point{3, Float64}(translation(inv(transform_from_base_to_wrist)))
settransform!(arrow_buoy_trans, pt, Vec{3, Float64}(wrist_buoy_transformed.v))

setelement!(mvis_alpha, default_frame(wrist_body))

# wrench_2 = Wrench(angular, linear)

# wrench_transformed = transform(state_alpha, wrench_2, default_frame(wrist_body))

# hydro_wrenches = Dict()
# hydro_wrenches(shoulder_ID) => wrench_test
# println(typeof(default_frame(findbody(mechanism_alpha, "r5m_shoulder_link"))))
# println(typeof(wrench_2))
# println(wrench_transformed)

# hydro_wrenches = Dict()
# hydro_wrenches[shoulder_ID] = wrench_2

# hydro_wrenches = Dict{BodyID, Wrench}(
#     BodyID(findbody(mechanism_alpha, "r5m_shoulder_link")) => transform(state_alpha, wrench_test, base_frame)
# )

after_r5m_joint2
r5m_wrist_link
[0.17551223196262425, 0.0958929423833131, -5.706889148349053e-6]


MeshCat Visualizer with path /meshcat/alpha/world/r5m_shoulder_link/r5m_upper_arm_link/r5m_forearm_link/r5m_wrist_link/after_r5m_joint2/<element> at http://127.0.0.1:8706

In [56]:
# ------------------------------------------------------------------------
#                              SIMULATION
# ------------------------------------------------------------------------
final_time = 0.1
# ts, qs, vs = simulate_with_ext_forces(state_alpha, final_time, hydro_calc!, simple_control!; Δt = 1e-2)
ts, qs, vs = simulate(state_alpha, final_time, simple_control!; Δt = 1e-2)

println("Simulation finished.")

Simulation finished.


In [57]:
# ------------------------------------------------------------------------
#                             VISUALIZATION
# ------------------------------------------------------------------------
# vis = Visualizer()
# open(vis)

# delete!(vis)
# mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf_file))
# set_configuration!(mvis, [0.0, 0.0])
# open(mvis)
MeshCatMechanisms.animate(mvis_alpha, ts, qs; realtimerate = 1.)

# animation = Animation(mvis, ts, qs)
# setanimation!(mvis, animation)

println("\n done.")


 done.
